In [ ]:
!pip install pandas
!pip install scikit-learn
!pip install mediapipe

## Linear regression

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
# Fetch the data using sklearn function
bunch = fetch_california_housing(download_if_missing=True, as_frame=True)

# Load the dataframe and view
df = bunch.frame
df.head()

In [ ]:
df = bunch.frame
x = df.iloc[:,:-1] # Select all the columns, except the last column
y = df.iloc[:,-1:] # Select the last column
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.33, random_state = 1)

input_scalar = StandardScaler()
output_scalar = StandardScaler()

x_train = input_scalar.fit_transform(x_train).T # Normalize train data
x_test = input_scalar.transform(x_test).T # Only transform test data using values of train data

y_train = output_scalar.fit_transform(y_train).reshape(-1)
y_test = output_scalar.transform(y_test).reshape(-1)

dataset_copy = [ x_train.copy(), x_test.copy(),  y_train.copy(),  y_test.copy()]

# Linear Regression Model -Supervised Learning

Now we define our linear regression model from scratch.

A linear regression model is of the form:

$y = a_1 x_1 + a_2 x_2 + \dots + a_nx_n + a_{n+1}$
  
The above can be rewritten using matrix multiplication as

$ y = w^T x $

where 

$ w = [a_1, a_2, \dots,  a_n, a_{n+1}]^T $

$ x = [x_1, x_2, \dots,  x_n]^T $


![lr](https://user-images.githubusercontent.com/81376570/122819892-02032a80-d2a9-11eb-993b-a3069f9df662.gif)

In [ ]:
class LinearRegression():
    def __init__(self, dim, lr = 0.1):
        assert isinstance
        self.lr = lr
        self.w = np.zeros((dim))
        self.grads = {"dw": np.zeros((dim)) +5}

    def forward(self, x):
        y = self.w.T @ x
        return y
  
    def backward(self, x, y_hat, y):
        assert y_hat.shape == y.shape
        self.grads["dw"] = (1 / x.shape[1]) * ((y_hat - y) @ x.T).T
        assert self.grads["dw"].shape == self.w.shape
    
    # print(self.grads["dw"])
    def optimize(self):
        self.w = self.w - self.lr * self.grads["dw"]

In [ ]:
num_epochs = 10000
train_loss_history = []
test_loss_history = []
w_history = []
dim = x_train.shape[0]
num_train = x_train.shape[1]
num_test = x_test.shape[1]


model = LinearRegression(dim = dim, lr = 0.1)
for i in range(num_epochs):
    y_hat = model.forward(x_train)
    train_loss = 1/(2 * num_train) * ((y_train - y_hat) ** 2).sum()
    w_history.append(model.w)
    model.backward(x_train,y_hat,y_train)
    model.optimize()

    y_hat = model.forward(x_test)
    test_loss = 1/(2 * num_test) * ((y_test - y_hat) ** 2).sum()

    train_loss_history.append(train_loss)
    test_loss_history.append(test_loss)

    if i % 20 == 0:
        print(f"Epoch {i} | Train Loss {train_loss} | Test Loss {test_loss}")

#plt.plot(range(num_epochs), train_loss_history, label = "Training")
#plt.plot(range(num_epochs), test_loss_history, label = "Test")
#plt.legend()
#plt.show()



In [ ]:
from sklearn.linear_model import LinearRegression as LR
from sklearn.metrics import mean_squared_error

x_train, x_test, y_train, y_test = dataset_copy
lr = LR()
lr.fit(x_train.T, y_train)
y_hat = lr.predict(x_test.T)
y_test = output_scalar.inverse_transform(y_test[np.newaxis,:])
y_hat  = output_scalar.inverse_transform(y_hat[np.newaxis,:])
error = mean_squared_error(y_test, y_hat, squared = True)
print("Test Set Error", error)

## Clustering -Unsupervised Learning

In [ ]:
from numpy import unique
from numpy import where
from sklearn.datasets import make_classification
from sklearn.cluster import Birch
from matplotlib import pyplot
# define dataset
X, _ = make_classification(n_samples=1000, n_features=2, n_informative=2, n_redundant=0, n_clusters_per_class=1, random_state=4)
# define the model
model = Birch(threshold=0.01, n_clusters=2)
# fit the model
model.fit(X)
# assign a cluster to each example
yhat = model.predict(X)
# retrieve unique clusters
clusters = unique(yhat)
# create scatter plot for samples from each cluster
for cluster in clusters:
    # get row indexes for samples with this cluster
    row_ix = where(yhat == cluster)
    # create scatter of these samples
    pyplot.scatter(X[row_ix, 0], X[row_ix, 1])
# show the plot
pyplot.show()

## Reinforcement learning

![RL](https://pytorch.org/tutorials/_images/mario.gif)
![rl](http://ai.stanford.edu/blog/assets/img/posts/2019-09-05-acteach/task-path-place.gif)
![rl](https://www.pair.toronto.edu/assets/img/teaser/graspd-eccv22.gif)
![rl](https://huggingface.co/blog/assets/78_deep_rl_dqn/atari-envs.gif)

In [ ]:
#pip install gym-notebook-wrapper

In [ ]:
#!pip3 install gym


In [ ]:
'''import gym
import math
import numpy as np
import tempfile
from gym import wrappers
from IPython import display
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
tdir = tempfile.mkdtemp()

env = gym.make('FrozenLake-v1')
np.random.seed(56776)
q_learning_table = np.zeros([env.observation_space.n,env.action_space.n])

# -- hyper --
num_epis = 500
num_iter = 200
learning_rate = 0.2
discount = 0.9

# -- training the agent ----
for epis in range(num_epis):
    
    state = env.reset()

    for iter in range(num_iter):
        action = np.argmax(q_learning_table[state,:] + np.random.randn(1,4))
        state_new,reward,done,_ = env.step(action)
        q_learning_table[state,action] = (1-learning_rate)* q_learning_table[state,action] + \
                                         learning_rate * (reward + discount*np.max(q_learning_table[state_new,:]) )
        state = state_new

        if done: break

print(np.argmax(q_learning_table,axis=1))
print(np.around(q_learning_table,5))
print('-------------------------------')
s = env.reset()
for _ in range(500):
    action  = np.argmax(q_learning_table[s,:])
    state_new,_,done,_ = env.step(action)
    env.render()
    s = state_new
    if done: break
    
''';

## YOLO - Object detection

In [ ]:
## Will take sometime to download torch 730mb
#!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -r requirements.txt  # install

In [ ]:
!python3 detect.py --source 0

## Facial Landmark Detection - DEMO

In [ ]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

In [ ]:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture(0)
with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
            continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(image)

        # Draw the face mesh annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_tesselation_style())
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_contours_style())
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_IRISES,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_iris_connections_style())
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Face Mesh', cv2.flip(image, 1))
        if cv2.waitKey(5) & 0xFF == 27:
            break
cap.release()